In [7]:
%load_ext autoreload
%autoreload 2

import os
import random
import pprint

import pysam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.append('/home/users/pjh/scripts/python_genome_package_dev/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import handygenome.common as common
from handygenome.common import ChromDict

FASTA_HG19 = common.DEFAULT_FASTAS['hg19']
FASTA_HG38 = common.DEFAULT_FASTAS['hg38']
CHROMDICT_HG19 = common.DEFAULT_CHROMDICTS['hg19']
CHROMDICT_HG38 = common.DEFAULT_CHROMDICTS['hg38']

# COSMIC file download

In [6]:
import handygenome.cosmic_download as cosmic_download
import handygenome.dbconv.cosmicdb as dbconv_cosmic

[W::hts_idx_load3] The index file is older than the data file: /home/users/pjh/scripts/python_genome_packages/data/popfreq/dbSNP_b155_GRCh37.p13.vcf.gz.csi


In [45]:
basename = 'CosmicFusionExport.tsv.gz'
cosmic_download.download_cosmic_file(
    f'/home/users/pjh/scripts/python_genome_package_dev/data/cosmic/v97/grch37/{basename}',
    email='p69180@kaist.ac.kr',
    password='Zhzhfl_231',
    cosmicfile=basename,
    cosmicver='v97',
    refver='GRCh37',
)


# SV-related

In [47]:
df_bkpt = pd.read_table(
    '/home/users/pjh/scripts/python_genome_package_dev/data/cosmic/v97/grch37/CosmicBreakpointsExport.tsv.gz',
    low_memory=False,
)
df_struct = pd.read_table(
    '/home/users/pjh/scripts/python_genome_package_dev/data/cosmic/v97/grch37/CosmicStructExport.tsv.gz',
    low_memory=False,
)
df_fusion = pd.read_table(
    '/home/users/pjh/scripts/python_genome_package_dev/data/cosmic/v97/grch37/CosmicFusionExport.tsv.gz',
    low_memory=False,
)

# Actionability

In [9]:
actionability_df = pd.read_table(
    '/home/users/pjh/scripts/python_genome_package_dev/data/cosmic/v97/grch37/ActionabilityData.tsv',
    low_memory=False,
)

In [24]:
pd.options.display.max_columns = None

In [12]:
actionability_df.columns

Index(['GENE', 'MUTATION_REMARK', 'GENOMIC_MUTATION_ID', 'FUSION_ID',
       'MUTATION_AA_SYNTAX', 'DISEASE', 'ACTIONABILITY_RANK',
       'DEVELOPMENT_STATUS', 'DRUG_COMBINATION', 'TESTING_REQUIRED',
       'TRIAL_STATUS', 'TRIAL_ID', 'TRIAL_NAME', 'COMPLETION_STATUS',
       'TRIAL_PRIMARY_COMPLETION_DATE', 'PATIENT_PRESCREENING',
       'RESULTS_AVAILABILITY', 'PROGRESSION_REMARK', 'SOURCE_TYPE', 'SOURCE',
       'TREATMENT_HISTORY', 'PATIENT_AGE', 'PRIMARY_OUTCOME_MEASURE',
       'NUMBER_OF_PATIENTS', 'TREATED_NUMBER', 'CONTROL_NUMBER',
       'CONTROL_TREATMENT', 'ORR_TREAT', 'ORR_CON', 'ORR_CI', 'ORR_PVAL',
       'DOR_TREAT', 'DOR_CON', 'DOR_CI', 'DOR_PVAL', 'PFS_TREAT', 'PFS_CON',
       'PFS_HR', 'PFS_CI', 'PFS_PVAL', 'TTP_TREAT', 'TTP_CON', 'TTP_HR',
       'TTP_CI', 'TTP_PVAL', 'DCR_TREAT', 'DCR_CON', 'DCR_PVAL', 'OS_TREAT',
       'OS_CON', 'OS_HR', 'OS_CI', 'OS_PVAL', 'OBJ_RR_TREAT', 'CR_COUNT',
       'PR_COUNT', 'SD_COUNT', 'RFS_TREAT', 'RFS_CON', 'RFS_HR', 'RFS_CI',
  

In [27]:
actionability_df.loc[~actionability_df['FUSION_ID'].isna()]

,GENE,MUTATION_REMARK,GENOMIC_MUTATION_ID,FUSION_ID,MUTATION_AA_SYNTAX,DISEASE,ACTIONABILITY_RANK,DEVELOPMENT_STATUS,DRUG_COMBINATION,TESTING_REQUIRED,TRIAL_STATUS,TRIAL_ID,TRIAL_NAME,COMPLETION_STATUS,TRIAL_PRIMARY_COMPLETION_DATE,PATIENT_PRESCREENING,RESULTS_AVAILABILITY,PROGRESSION_REMARK,SOURCE_TYPE,SOURCE,TREATMENT_HISTORY,PATIENT_AGE,PRIMARY_OUTCOME_MEASURE,NUMBER_OF_PATIENTS,TREATED_NUMBER,CONTROL_NUMBER,CONTROL_TREATMENT,ORR_TREAT,ORR_CON,ORR_CI,ORR_PVAL,DOR_TREAT,DOR_CON,DOR_CI,DOR_PVAL,PFS_TREAT,PFS_CON,PFS_HR,PFS_CI,PFS_PVAL,TTP_TREAT,TTP_CON,TTP_HR,TTP_CI,TTP_PVAL,DCR_TREAT,DCR_CON,DCR_PVAL,OS_TREAT,OS_CON,OS_HR,OS_CI,OS_PVAL,OBJ_RR_TREAT,CR_COUNT,PR_COUNT,SD_COUNT,RFS_TREAT,RFS_CON,RFS_HR,RFS_CI,RFS_PVAL,BLOOD_RESPONSE,RESPONSE_VALUE,TIMEPOINT,CLASSIFICATION_ID,LAST_UPDATED
560,BCR,BCR-ABL ?:? COSF1783,NaN,1783.0,NaN,NS / other / neoplasm,3,Phase 1,Dasatinib,NaN,Completed,NCT00316953,NaN,ECD,2009Sep01,No,Yes,NaN,1,21263099,Previously-treated,NaN,Time to progression,28.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSO31314888,2022Aug04
561,BCR,BCR-ABL ?:? COSF1783,NaN,1783.0,NaN,NS / other / neoplasm,3,Phase 2,Ponatinib,NaN,Recruiting,NCT03934372,NaN,ECD,2024Oct08,Yes,No,NaN,4,NCT03934372,Previously-treated,NaN,Overall Response Rate,60.0,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSO31314888,2022Aug04
562,BCR,BCR-ABL ?:? COSF1783,NaN,1783.0,NaN,NS / other / neoplasm,3,Phase 2,Bosutinib,NaN,Recruiting,NCT03297606,CAPTUR group 2,ECD,2022Sep30,Yes,No,NaN,4,NCT03297606,NaN,NaN,Overall Response Rate,720.0,720.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSO31314888,2022Jan24
563,BCR,BCR-ABL ?:? COSF1783,NaN,1783.0,NaN,NS / other / neoplasm,3,Phase 2,Ponatinib,NaN,Recruiting,NCT04591431,ROME,ECD,2024Apr01,Yes,No,NaN,4,NCT04591431,NaN,NaN,Overall Response Rate,384.0,384.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSO31314888,2021Aug25
564,BCR,BCR-ABL ?:? COSF1783,NaN,1783.0,NaN,NS / other / neoplasm,3,Unknown,Bosutinib,NaN,"Active, not recruiting",NCT01903733,NaN,ECD,2020Jun01,Yes,No,NaN,4,NCT01903733,NaN,NaN,Not defined,282.0,282.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSO31314888,2021Jul16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1139,BCR,BCR-ABL ?:? COSF1783,NaN,1783.0,NaN,haematopoietic and lymphoid tissue / lymphoid ...,1,Unknown,Dasatinib + Prednisone + Vincristine sulphate,NaN,Recruiting,NCT05026229,NaN,ECD,2023Jun30,Yes,No,NaN,4,NCT05026229,Untreated,NaN,"Residual disease, Residual cancer burden",60.0,NaN,NaN,Cytarabine + Dasatinib + Methotrexate + Predni...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSO27984916,2021Nov22
1140,BCR,BCR-ABL e13a2 (b2a2) COSF1757,NaN,1757.0,NaN,haematopoietic and lymphoid tissue / haematopo...,3,Out of trials human study,Imatinib,NaN,Unknown,NaN,NaN,NaN,NaN,Yes,Yes,NaN,1,19714331,NaN,NaN,Unknown,34.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Complete cytogenetic,58.8,NaN,COSO27985072,2021May18
1141,BCR,BCR-ABL e14a2 (b3a2) COSF1755,NaN,1755.0,NaN,

In [26]:
actionability_df['TESTING_REQUIRED'].unique()

array([nan, 'Required', 'Not required', 'Expression', 'wt KRAS',
       'HER2 expression'], dtype=object)

In [4]:
mutexp_path = '/home/users/pjh/scripts/python_genome_packages/data/cosmic/v96/grch37/CosmicMutantExport.tsv.gz'
ncv_path = '/home/users/pjh/scripts/python_genome_packages/data/cosmic/v96/grch37/CosmicNCV.tsv.gz'

In [5]:
is_hg19 = True

In [6]:
summary = dict()
site_count = dict()

In [7]:
cosmicdb.load_datafile(ncv_path, cosmicdb.line_parser_ncv, summary, site_count, logging_lineno=1_000_000)

[KST 2022-08-03 17:16:45 INFO] COSMIC converter: processing 1,000,000th line
[KST 2022-08-03 17:17:25 INFO] COSMIC converter: processing 2,000,000th line
[KST 2022-08-03 17:18:04 INFO] COSMIC converter: processing 3,000,000th line
[KST 2022-08-03 17:18:51 INFO] COSMIC converter: processing 4,000,000th line
[KST 2022-08-03 17:19:38 INFO] COSMIC converter: processing 5,000,000th line
[KST 2022-08-03 17:20:41 INFO] COSMIC converter: processing 6,000,000th line
[KST 2022-08-03 17:21:13 INFO] COSMIC converter: processing 7,000,000th line
[KST 2022-08-03 17:22:30 INFO] COSMIC converter: processing 8,000,000th line
[KST 2022-08-03 17:23:54 INFO] COSMIC converter: processing 9,000,000th line
[KST 2022-08-03 17:24:43 INFO] COSMIC converter: processing 10,000,000th line
[KST 2022-08-03 17:25:38 INFO] COSMIC converter: processing 11,000,000th line
[KST 2022-08-03 17:27:46 INFO] COSMIC converter: processing 12,000,000th line
[KST 2022-08-03 17:28:27 INFO] COSMIC converter: processing 13,000,000th 

In [8]:
cosmicdb.load_datafile(mutexp_path, cosmicdb.line_parser_mutantexport, summary, site_count, logging_lineno=1_000_000)

[KST 2022-08-03 17:35:31 INFO] COSMIC converter: processing 1,000,000th line
[KST 2022-08-03 17:35:31 INFO] COSMIC converter: processing 1,000,000th line
[KST 2022-08-03 17:36:10 INFO] COSMIC converter: processing 2,000,000th line
[KST 2022-08-03 17:36:10 INFO] COSMIC converter: processing 2,000,000th line
[KST 2022-08-03 17:36:45 INFO] COSMIC converter: processing 3,000,000th line
[KST 2022-08-03 17:36:45 INFO] COSMIC converter: processing 3,000,000th line
[KST 2022-08-03 17:37:21 INFO] COSMIC converter: processing 4,000,000th line
[KST 2022-08-03 17:37:21 INFO] COSMIC converter: processing 4,000,000th line
[KST 2022-08-03 17:37:56 INFO] COSMIC converter: processing 5,000,000th line
[KST 2022-08-03 17:37:56 INFO] COSMIC converter: processing 5,000,000th line
[KST 2022-08-03 17:38:31 INFO] COSMIC converter: processing 6,000,000th line
[KST 2022-08-03 17:38:31 INFO] COSMIC converter: processing 6,000,000th line
[KST 2022-08-03 17:39:06 INFO] COSMIC converter: processing 7,000,000th line

In [9]:
cosmicdb.modify_summary(summary, site_count, FASTA_HG19, is_hg19, logging_lineno=10_000)

[KST 2022-08-03 18:08:53 INFO] COSMIC converter: Beginning SUMMARY MODIFICATION
[KST 2022-08-03 18:08:53 INFO] COSMIC converter: Beginning SUMMARY MODIFICATION
[KST 2022-08-03 18:09:16 INFO] COSMIC converter: 10,000th COSV ID processed
[KST 2022-08-03 18:09:16 INFO] COSMIC converter: 10,000th COSV ID processed
[KST 2022-08-03 18:09:34 INFO] COSMIC converter: 20,000th COSV ID processed
[KST 2022-08-03 18:09:34 INFO] COSMIC converter: 20,000th COSV ID processed
[KST 2022-08-03 18:09:42 INFO] COSMIC converter: 30,000th COSV ID processed
[KST 2022-08-03 18:09:42 INFO] COSMIC converter: 30,000th COSV ID processed
[KST 2022-08-03 18:09:50 INFO] COSMIC converter: 40,000th COSV ID processed
[KST 2022-08-03 18:09:50 INFO] COSMIC converter: 40,000th COSV ID processed
[KST 2022-08-03 18:10:00 INFO] COSMIC converter: 50,000th COSV ID processed
[KST 2022-08-03 18:10:00 INFO] COSMIC converter: 50,000th COSV ID processed
[KST 2022-08-03 18:10:08 INFO] COSMIC converter: 60,000th COSV ID processed
[KST

In [10]:
vcfspec_cosv_map = cosmicdb.get_vcfspec_cosv_map(summary, FASTA_HG19, is_hg19)

[KST 2022-08-03 19:01:19 INFO] COSMIC converter: Beginning VCFSPEC-TO-COSV MAPPING
[KST 2022-08-03 19:01:19 INFO] COSMIC converter: Beginning VCFSPEC-TO-COSV MAPPING
[KST 2022-08-03 19:05:30 INFO] COSMIC converter: Finished VCFSPEC-TO-COSV MAPPING
[KST 2022-08-03 19:05:30 INFO] COSMIC converter: Finished VCFSPEC-TO-COSV MAPPING


In [15]:
sorted_vcfspecs = cosmicdb.sort_vcfspecs(vcfspec_cosv_map, CHROMDICT_HG19)

[autoreload of handygenome.dbconv.cosmicdb failed: Traceback (most recent call last):
  File "/home/users/pjh/tools/miniconda/210821/miniconda3/envs/genome_v5/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/users/pjh/tools/miniconda/210821/miniconda3/envs/genome_v5/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/home/users/pjh/tools/miniconda/210821/miniconda3/envs/genome_v5/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/home/users/pjh/tools/miniconda/210821/miniconda3/envs/genome_v5/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 289, in update_function
    setattr(old, name, getattr(new, name))
ValueError: write_outfile() requires a code object with 3 free vars, not 0
]
[KST 2022-08-03 19:27:04 INFO] COSMIC converter: B

In [16]:
outfile_path = '/home/users/pjh/scripts/python_genome_packages/data/cosmic/v96/grch37/nonSV.vcf.gz'
chromdict = CHROMDICT_HG19
refver = 'GRCh37'
cosmic_version = 'v96'

cosmicdb.write_outfile(outfile_path, summary, site_count, vcfspec_cosv_map, sorted_vcfspecs, chromdict, refver, cosmic_version)

[W::bcf_update_info] INFO/END=10107 is smaller than POS at 1:10108
[KST 2022-08-03 19:28:18 INFO] COSMIC converter: 100,000 lines written
[KST 2022-08-03 19:28:18 INFO] COSMIC converter: 100,000 lines written
[KST 2022-08-03 19:28:18 INFO] COSMIC converter: 100,000 lines written
[KST 2022-08-03 19:28:18 INFO] COSMIC converter: 100,000 lines written
[KST 2022-08-03 19:28:18 INFO] COSMIC converter: 100,000 lines written
[KST 2022-08-03 19:28:18 INFO] COSMIC converter: 100,000 lines written
[KST 2022-08-03 19:28:24 INFO] COSMIC converter: 200,000 lines written
[KST 2022-08-03 19:28:24 INFO] COSMIC converter: 200,000 lines written
[KST 2022-08-03 19:28:24 INFO] COSMIC converter: 200,000 lines written
[KST 2022-08-03 19:28:24 INFO] COSMIC converter: 200,000 lines written
[KST 2022-08-03 19:28:24 INFO] COSMIC converter: 200,000 lines written
[KST 2022-08-03 19:28:24 INFO] COSMIC converter: 200,000 lines written
[KST 2022-08-03 19:28:30 INFO] COSMIC converter: 300,000 lines written
[KST 2022-

In [17]:
for cosv in summary.keys():
    break
    
print(cosv)
print(summary[cosv])

COSV64950244
{'primary_site': {'endometrium': 1}, 'primary_site_somatic': {'endometrium': 1}, 'hgvsg': {'10:g.79556160A>G'}, 'hgvsc': {'ENST00000372391.2:c.5308+49T>C', 'ENST00000372388.2:c.4288+49T>C'}, 'coding_score': None, 'noncoding_score': None, 'vcfspec': <Vcfspec (10:79556160 A>G)>}
